## Purpose

This notebook processes the input data for the simulation of an Australian distribution network feeder. Load and generation data is sourced from the [Pecan Street](https://www.pecanstreet.org/) student-licensed data set. Power factor data is sourced from the [ECO](https://www.vs.inf.ethz.ch/res/show.html?what=eco-data) data set and combined with the Pecan Street data to produce active and reactive power load profiles.

Network models from the [Australian Low Voltage Feeder Taxonomy](https://near.csiro.au/assets/f325fb3c-2dcd-410c-97a8-e55dc68b8064) are converted from OpenDSS format to pandapower. 

The overall data cleaning process is depicted below:

![data_cleaning](https://i.ibb.co/2KZ48xg/data-cleaning.png)



## Load Data Processing
The load data was downloaded from the Pecan Street Dataport. The original files were called `1s_data_austin_file{1,2,3,4}.csv.gz`. 


Each is unzipped and processed. The single file containing all households is split into a file per household containing only the `dataid`, `localminute`, `grid`, `solar` and `solar2` columns. This reduces the size of the data set, and makes it practical to manipulate the data from a single household at once in pandas.


In [4]:
from pathlib import Path
import pandas as pd
import numpy as np
import csv
import os
from datetime import datetime
from multiprocessing.pool import ThreadPool
from scipy import io, stats
import math
from typing import Tuple, Any
from datetime import datetime, timedelta
import random


In [5]:
PECAN_ST_FILENAMES = [
    "1s_data_austin_file1.csv.gz",
    "1s_data_austin_file2.csv.gz",
    "1s_data_austin_file3.csv.gz",
    "1s_data_austin_file4.csv.gz",
]

INPUT_DATA_PATH = Path("./input_data")
OUTPUT_DATA_PATH = Path("./output_data")

# Used for creating test and training sets later
# Dates sourced from: https://www.calendardate.com/year2018.php
SPRING_START = datetime(2018, 3, 20, 0, 0, 0)
SPRING_END = datetime(2018, 6, 20, 0, 0, 0)

SUMMER_START = datetime(2018, 6, 21, 0, 0)
SUMMER_END = datetime(2018, 9, 21, 0, 0)

AUTUMN_START = datetime(2018, 9, 22, 0, 0)
AUTUMN_END = datetime(2018, 12, 20, 0, 0)

START_OF_YEAR = datetime(2018, 1, 1, 0, 0)
WINTER_START = datetime(2018, 12, 21, 0, 0)
END_OF_YEAR = datetime(2018, 12, 31, 0, 0)

In [6]:
def create_household_file(file_path: Path, header_row: str):
    with open(file_path, "w") as fd:
        fd.write(header_row)


def process_file(file_path: Path):
    file_descriptors = {}
    header_row = "dataid,localminute,grid,solar,solar2\n"
    try:
        with open(file_path, "r") as infile:
            csv_reader = csv.DictReader(infile)
            for row in csv_reader:
                dataid = row["dataid"]
                localminute = row["localminute"]
                grid = row["grid"]
                solar = row["solar"]
                solar2 = row["solar2"]

                file_descriptor = file_descriptors.get(dataid, None)
                if not file_descriptor:
                    output_file_path = OUTPUT_DATA_PATH / f"{dataid}-load-pv.csv"
                    if not output_file_path.exists():
                        create_household_file(output_file_path, header_row)
                    file_descriptor = open(output_file_path, "a")
                    file_descriptors[dataid] = file_descriptor
                file_descriptor.write(
                    f"{dataid},{localminute},{grid},{solar},{solar2}\n",
                )
    finally:
        for fd in file_descriptors.values():
            fd.close()


def unzip_raw_data(data_path: Path, unzip_path: Path):
    if not unzip_path.exists():
        # Lord forgive me for this but Python is terribly slow at this
        os.system(f"gzip -d < {data_path.resolve()} > {unzip_path.resolve()}")
        print("unzipped")
        


def process_data(data_path: Path):
    outfile_name = data_path.stem
    unzipped_path = INPUT_DATA_PATH / outfile_name
    unzip_raw_data(data_path, unzipped_path)
    process_file(unzipped_path)
    unzipped_path.unlink()


In [27]:
# NOTE: This takes a long time to run, 1.5 hours on an M1 mac.
# I could probably optimise it, but I'm only running it once 
# Should probably also multi thread it
# for file_name in PECAN_ST_FILENAMES:
#     print(file_name)
#     data_path = INPUT_DATA_PATH / file_name
#     process_data(data_path)



## Aggregate to 30s resolution
There's still a lot of data to work with. Though a resolution of 1s is interesting, the volume of data is too large for us to work with sensibly. Let's downsample the data to 30s resolution to make it a bit easier to work with.

In [7]:
def get_closest_interval(dt):
    seconds = dt.second
    interval_args = {
        "year": dt.year,
        "month": dt.month,
        "day": dt.day,
        "hour": dt.hour,
        "minute": dt.minute,
    }

    if seconds < 30:
        return datetime(**interval_args, second=0)
    else:
        return datetime(**interval_args, second=30)


def calculate_use(grid: str, solar: str, solar2: str) -> Union[float, None]:
    if not grid:
        return None
    grid_val = float(grid)
    solar_val = float(solar) if solar else 0
    solar2_val = float(solar2) if solar2 else 0

    # This may seem weird but it's explained here: https://docs.google.com/document/d/1_9H9N4cgKmJho7hK8nii6flIGKPycL7tlWEtd4UhVEQ/edit#
    return grid_val + solar_val + solar2_val


def calculate_net_solar(solar: str, solar2: str) -> Union[float, None]:
    if not solar and not solar2:
        return None
    solar_val = float(solar) if solar else 0
    solar2_val = float(solar) if solar2 else 0

    return solar_val + solar2_val


def downsample_data(file_path: Path, out_path: Path):
    data_periods = {}
    with open(file_path, "r") as infile:
        csv_reader = csv.DictReader(infile)
        for line in csv_reader:
            # The last three characters are TZ info, we will lose an hour's data every time the clocks change.
            # Fortunately, we don't really care about that
            localminute = line["localminute"][:-3]
            grid = line["grid"]
            solar = line["solar"]
            solar2 = line["solar2"]
            solar_val = calculate_net_solar(solar, solar2)
            use = calculate_use(grid, solar, solar2)
            dt = datetime.strptime(localminute, "%Y-%m-%d %H:%M:%S")

            interval = get_closest_interval(dt)
            if not data_periods.get(interval, None):
                data_periods[interval] = dict(
                    solar_sum=0, solar_count=0, use_sum=0, use_count=0
                )

            data = data_periods[interval]
            if solar_val is not None:
                data["solar_sum"] += solar_val
                data["solar_count"] += 1
            if use is not None:
                data["use_sum"] += use
                data["use_count"] += 1

    intervals = sorted(data_periods)
    with open(out_path, "w") as outfile:
        header = "datetime,solar,use\n"
        outfile.write(header)
        for interval in intervals:
            interval_data = data_periods[interval]
            iso_date = interval.isoformat()
            solar_count = interval_data["solar_count"]
            solar_sum = interval_data["solar_sum"]

            use_count = interval_data["use_count"]
            use_sum = interval_data["use_sum"]

            solar_val = ""
            use_val = ""
            if solar_count:
                solar_val = solar_sum / solar_count
            if use_count:
                use_val = use_sum / use_count
            outfile.write(f"{iso_date},{solar_val},{use_val}\n")


In [18]:
# NOTE: This also takes ages - should really have written this more effficiently
threads = 25
t = ThreadPool(threads)

files_to_process = []
for file_path in OUTPUT_DATA_PATH.iterdir():
    if not str(file_path).endswith("load-pv.csv"):
        continue
    outfile_name = f"{file_path.stem}-reduced.csv"
    outfile_path = file_path.parent / outfile_name
    files_to_process.append((
        file_path,
        outfile_path
    ))
 

with ThreadPool(threads) as t:
    t.starmap(downsample_data, files_to_process)


## Further processing
The next steps are:
1. Aggregate the ECO dataset and compute an average power factor.
2. Check the existing load and PV data for outliers and replace them with neighbouring values. We know from experience that these values exist. 
3. Perturb the average power factor by +/- 10% and compute the appropriate reactive power for each interval in the load data.
4. Bootstrap to 100 load and PV profiles
5. Create active, reactive, and pv profiles for all the households.



In [17]:
# The input data was requested from ETH Zurich and downloaded directly from their portal
adres_input_path = INPUT_DATA_PATH / "ADRES_Daten_120208.mat"
loaded = io.loadmat(adres_input_path)
adres_df = pd.DataFrame(loaded["Data"]["PQ"][0][0])


In [18]:
adres_df

,0,1,2,3,4,5,6,7,8,9,...,170,171,172,173,174,175,176,177,178,179
0,40.200001,-14.670000,22.150000,3.189000,141.500000,-80.730003,550.083313,-31.236666,247.800003,66.066666,...,168.100006,20.600000,10.700000,-0.995,55.000000,-31.100000,52.200001,-42.900002,35.000000,-1.700000
1,41.500000,-14.980000,22.180000,3.529000,140.699997,-80.550003,549.080017,-31.166000,247.960007,66.199997,...,168.500000,20.600000,10.640000,-0.947,54.900002,-31.180000,52.500000,-43.000000,35.000000,-1.800000
2,40.799999,-15.120000,22.040001,3.408000,141.000000,-80.330002,549.099976,-31.172001,247.880005,65.839996,...,168.100006,20.600000,10.810000,-0.945,55.799999,-31.299999,52.900002,-43.000000,35.000000,-1.800000
3,40.900002,-14.660000,22.219999,2.956000,141.100006,-80.550003,540.880005,-31.386000,247.399994,65.739998,...,168.500000,20.600000,10.710000,-0.962,60.200001,-31.660000,52.500000,-42.900002,35.000000,-1.800000
4,40.700001,-14.860000,22.120001,3.148000,141.000000,-80.669998,428.760010,-29.719999,247.259995,65.699997,...,168.399994,20.700001,10.650000,-0.927,60.200001,-31.740000,52.500000,-42.900002,35.099998,-1.900000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1209595,102.300003,24.510000,133.800003,-62.160000,108.300003,54.270000,290.899994,17.570000,107.699997,40.610001,...,139.899994,-9.500000,222.000000,2.300,57.500000,-7.800000,34.200001,-27.740000,28.400000,-27.469999
1209596,102.500000,24.459999,133.800003,-62.169998,113.400002,54.000000,289.799988,16.240000,107.699997,40.669998,...,139.800003,-9.500000,221.699997,2.300,57.200001,-7.900000,34.200001,-27.889999,28.400000,-27.650000
1209597,102.699997,24.549999,133.800003,-62.349998,112.300003,54.090000,287.299988,13.430000,107.900002,40.720001,...,139.800003,-9.600000,221.800003,2.400,56.299999,-7.700000,34.200001,-27.820000,28.400000,-27.600000
1209598,102.500000,24.389999,134.000000,-62.290001,108.199997,54.189999,282.299988,11.320000,107.599998,40.020000,...,139.899994,-9.500000,221.500000,2.400,56.500000,-7.700000,34.299999,-27.180000,28.500000,-27.740000


## Average power factor calculation

Each household in the data set has six columns - P1, Q1, P2, Q2, P3, Q3.

To compute the average power factor across the entire data set we will compute the average P across all the columns, and the average Q across all the columns. This can then be used to calculate an average power factor. 

As we can't make any assumptions about the relationship between the ADRES data set and the Pecan Street data this is all we will do.

We can then use the average power factor to finish processing the Pecan Street data.

In [19]:
# Even columns are P, odd are Q
p_cols = [col for col in adres_df if col % 2 == 0]
q_cols = [col for col in adres_df if col % 2 == 1]

# Sum across columns then down the column
p_sum = adres_df[p_cols].sum(axis=1).sum(axis=0)
q_sum = adres_df[q_cols].sum(axis=1).sum(axis=0)

# We've summed across the rows and the columns so need to divide by their length to get the average
avg_p = p_sum / len(p_cols) / len(adres_df)
avg_q = q_sum / len(q_cols) / len(adres_df)

avg_s = math.sqrt((avg_p**2 + avg_q**2))

print(f"Average P: {avg_p} W")
print(f"Average Q: {avg_q} VAr")
print(f"Average S: {avg_s} VA")

Average P: 215.26437624926513 W
Average Q: 27.710495002939446 VAr
Average S: 217.04060268828297 VA


In [22]:
avg_pf = avg_p / avg_s
print(f"Average PF: {avg_pf}")


Average PF: 0.9918161559771888


## Now we have a power factor to use as our baseline
Somewhat unsurprisingly, it's not far from unity (0.9918161559771888). In general, residential premises typically operate near a unity power factor.

Next step: use it to create feasible reactive power readings for each load and create the final input profiles.

We will need to do the following:
1. Filter out values that are 5 standard deviations from the mean, they are likely measurement errors and will distort the data. 
2. Fill in missing values. For this we will be replacing missing intervals with the closest interval at that time of day with a value.

In [24]:
MIN_DATE = datetime(2018, 1, 1, 0, 0, 0)
MAX_DATE = datetime(2018, 12, 31, 23, 59, 30)
AVG_SOLAR_SYSTEM_KW = 6.9


def add_missing_periods(df, freq="30S"):
    date_range = pd.date_range(MIN_DATE, MAX_DATE, freq=freq)
    df = df.reindex(date_range, fill_value=np.nan)
    df["datetime"] = df.index
    return df


def get_na_proportion(df) -> Any:
    solar_proportion = df["solar"].isna().sum() / len(df["solar"])
    use_proportion = df["use"].isna().sum() / len(df["use"])
    return {"solar": solar_proportion, "use": use_proportion}


def remove_outliers(df, column, sigma_threshold=5):
    outliers = np.abs(stats.zscore(df[column], nan_policy="omit")) > sigma_threshold
    df.loc[outliers, column] = np.nan
    # Only fill at max 5 intervals forward
    df[column] = df[column].fillna(method="ffill", limit=5)


def get_replacement_value(df, index, column):
    min_date = np.min(df["datetime"]).to_pydatetime()
    max_date = np.max(df["datetime"]).to_pydatetime()

    row = df.loc[index]

    row_datetime = row["datetime"].to_pydatetime()

    first_relevant_datetime = min_date.replace(
        hour=row_datetime.hour,
        minute=row_datetime.minute,
        second=row_datetime.second,
    )
    last_relevant_datetime = max_date.replace(
        hour=row_datetime.hour,
        minute=row_datetime.minute,
        second=row_datetime.second,
    )

    forward_dt = row_datetime
    back_dt = row_datetime

    replacement_val = np.nan

    while forward_dt <= last_relevant_datetime or back_dt >= first_relevant_datetime:
        if back_dt in df.index and not np.isnan(df.loc[back_dt, column]):
            replacement_val = df.loc[back_dt, column]
            break

        if forward_dt in df.index and not np.isnan(df.loc[forward_dt, column]):
            replacement_val = df.loc[forward_dt, column]
            break

        forward_dt += timedelta(days=1)
        back_dt -= timedelta(days=1)

    if np.isnan(replacement_val):
        raise ValueError(f"Couldn't find replacement values for date: {index}")

    return replacement_val


def replace_missing_values(df, column):
    indices_to_replace = df.loc[df[column].isna(), column].index
    replacement_values = {}
    # This is O(N^2) in the worst case (which could be pretty bad)
    # Fortunately for us our data is pretty complete
    for index in indices_to_replace:
        replacement_val = get_replacement_value(df, index, column)
        replacement_values[index] = replacement_val
    df[column] = df[column].fillna(value=replacement_values)


def normalise_solar(df):
    # Remove all values less than 0
    # Parasitics are important, but not *that* important
    negative_vals = df["solar"] < 0
    df.loc[negative_vals, "solar"] = 0
    abs_solar = np.abs(df["solar"])
    max_val = np.max(abs_solar)
    # Need to produce outputs in MW not kW 
    df["solar"] = (df["solar"] / max_val) * AVG_SOLAR_SYSTEM_KW / 1000


def add_reactive(df):
    power_factor = pd.Series([avg_pf] * len(df), index=df.index)
    random_perturbation = np.random.uniform(0.9, 1.0, size=len(df))
    power_factor *= random_perturbation
    real_power = df["use"]
    apparent_power = real_power / power_factor
    # Multiplication is faster than exponentiation here
    s_squared = apparent_power * apparent_power
    p_squared = real_power * real_power
    q_squared = s_squared - p_squared
    df["use_reactive"] = np.sqrt(q_squared)
    df.rename(columns={"use": "use_active"}, inplace=True)





def process_site_data(file_path) -> Tuple[pd.DataFrame, Any]:
    df = pd.read_csv(
        file_path,
        parse_dates=["datetime"],
        dtype={
            "use": float,
            "solar": float,
        },
    )
    df = df.set_index(df["datetime"])
    df = add_missing_periods(df)

    na_proportions = get_na_proportion(df)
    site_has_solar = na_proportions["solar"] != 1.0

    remove_outliers(df, "solar")
    remove_outliers(df, "use")

    if site_has_solar:
        replace_missing_values(df, "solar")
        normalise_solar(df)

    replace_missing_values(df, "use")
    # Convert use from kW to MW for pandapower profiles
    df["use"] = df["use"] / 1000

    add_reactive(df)
    df = df.reset_index(drop=True)
    return df, na_proportions


household_na_proportions = []
household_index_map = []

def create_agg_profiles(active: pd.DataFrame, reactive: pd.DataFrame, pv: pd.DataFrame):
    load_index = 0
    solar_index = 0
    for file_path in OUTPUT_DATA_PATH.iterdir():
        site_id = file_path.stem.split("-")[0]
        if not str(file_path).endswith("load-pv-reduced.csv"):
            continue

        df, na_proportions = process_site_data(file_path)

        site_has_solar = na_proportions["solar"] != 1.0

        reactive[load_index] = df["use_reactive"]
        active[load_index] = df["use_active"]

        if site_has_solar:
            pv[solar_index] = df["solar"]

        index_map = {"household_id": site_id, "load_index": load_index}

        load_index += 1
        if site_has_solar:
            index_map["solar_index"] = solar_index
            solar_index += 1

        household_index_map.append(index_map)
        household_na_proportions.append(
            {
                "household_id": site_id,
                "solar_na": na_proportions["solar"],
                "use_na": na_proportions["use"],
            }
        )


In [ ]:
# Set random seed for pertubing active power
random.seed(42)
reactive = pd.DataFrame()
active = pd.DataFrame()
pv = pd.DataFrame()


date_range = pd.date_range(MIN_DATE, MAX_DATE, freq="30S")
reactive["datetime"] = date_range
active["datetime"] = date_range
pv["datetime"] = date_range

# Pretty slow again, sorry
create_agg_profiles(active, reactive, pv)

In [26]:
# We need to keep track of which household is mapped to which index in the final data set
household_map_df = pd.DataFrame(household_index_map) 
na_proportion_df = pd.DataFrame(household_na_proportions)

household_map_df.to_csv(OUTPUT_DATA_PATH / "household_index_map.csv", index=False)
na_proportion_df.to_csv(OUTPUT_DATA_PATH / "na_proportion.csv", index=False)



## Bootstrapping time!
Now we need to bootstrap to the 109 profiles required for the LVFT models.

We will employ block bootstrapping, building up new load profiles by selecting a day of data at random from our original profiles

In [27]:
def is_int(val):
    try:
        int(val)
        return True
    except ValueError:
        return False


def get_days_data(df, date, column):
    lower_datetime = pd.to_datetime(date)
    upper_datetime = lower_datetime.replace(hour=23, minute=59, second=59)
    mask = (df.index >= lower_datetime) & (df.index < upper_datetime)
    return df.loc[mask, column]


def get_bootstrap_samples(df, n_profiles):
    df = df.set_index("datetime")
    profiles_to_sample_from = [col for col in df.columns if is_int(col)]
    samples_to_take = []
    dates_to_sample = sorted(df.index.map(pd.Timestamp.date).unique())
    for _ in range(n_profiles):
        sample_columns = random.choices(profiles_to_sample_from, k=len(dates_to_sample))
        samples_to_take.append(list(zip(dates_to_sample, sample_columns)))
    df = df.reset_index(drop=True)
    return samples_to_take


def bootstrap_samples(df, samples_to_take, first_index):
    df = df.set_index("datetime")
    idx = first_index
    for sample in samples_to_take:
        data_sample = None
        for date, sample_column in sample:
            data = get_days_data(df, date, sample_column)
            if data_sample is None:
                data_sample = data
            else:
                data_sample = pd.concat([data_sample, data])
        data_sample = data_sample.rename(idx)
        df = df.join(data_sample)
        idx += 1
    return df

In [28]:
# Set random seed for bootstrapping samples
random.seed(42)
load_bootstrap_samples = get_bootstrap_samples(reactive, 84)
reactive_df = bootstrap_samples(reactive, load_bootstrap_samples, 25)
active_df = bootstrap_samples(active, load_bootstrap_samples, 25)

pv_bootstrap_samples = get_bootstrap_samples(pv, 91)
pv_df = bootstrap_samples(pv, pv_bootstrap_samples, 19)


## Now to create a test-train split
 
Similar to the bootstrapping we will sample in blocks for the test-train split. Given that the data is time series it makes little intuitive sense to randomly sample rows. Instead 14 days are randomly selected from each season to create a test set.



In [29]:
def get_days_data(df, date, column):
    lower_datetime = pd.to_datetime(date)
    upper_datetime = lower_datetime.replace(hour=23, minute=59, second=59)
    mask = (df.index >= lower_datetime) & (df.index < upper_datetime)
    return df.loc[mask, column]


def get_dates_in_range(*args):
    if len(args) % 2 != 0 or not len(args):
        raise ValueError("Date ranges must be specified in pairs")
    date_range = []
    pairs = [(args[i], args[i + 1]) for i in range(0, len(args) - 1, 2)]
    for start, end in pairs:
        pair_range = list(pd.date_range(start, end, freq="1D"))
        date_range += pair_range

    return date_range


def sample_n_from_each_collection(collections, n):
    samples = []
    for collection in collections:
        samples += random.sample(collection, n)

    return samples


def create_test_train_split(df, test_dates):
    test_indices = None
    # We don't care which column we select, we just care about the index
    col_to_select = df.columns[0]
    for test_date in test_dates:
        days_data = get_days_data(df, test_date, col_to_select)
        if test_indices is None:
            test_indices = days_data
        else:
            test_indices = pd.concat([test_indices, days_data])
    test_indices = test_indices.index
    train_indices = ~df.index.isin(test_indices)
    test_df = df.loc[test_indices]
    train_df = df.loc[train_indices]
    return test_df, train_df


In [30]:
random.seed(42)

spring_dates = get_dates_in_range(SPRING_START, SPRING_END)
summer_dates = get_dates_in_range(SUMMER_START, SUMMER_END)
autumn_dates = get_dates_in_range(AUTUMN_START, AUTUMN_END)
winter_dates = get_dates_in_range(
    START_OF_YEAR, SPRING_START, WINTER_START, END_OF_YEAR
)

test_dates = sample_n_from_each_collection(
    [spring_dates, summer_dates, autumn_dates, winter_dates], 14
)

test_reactive_df, train_reactive_df = create_test_train_split(reactive_df, test_dates)
test_active_df, train_active_df = create_test_train_split(active_df, test_dates)
test_pv_df, train_pv_df = create_test_train_split(pv_df, test_dates)

In [31]:
# Reset the indices before saving
test_reactive_df = test_reactive_df.reset_index()
train_reactive_df = train_reactive_df.reset_index()

test_active_df = test_active_df.reset_index()
train_active_df = train_active_df.reset_index()

test_pv_df = test_pv_df.reset_index()
train_pv_df = train_pv_df.reset_index()


In [32]:
test_reactive_df.to_csv(OUTPUT_DATA_PATH / "test_reactive.csv", index=False)
train_reactive_df.to_csv(OUTPUT_DATA_PATH / "train_reactive.csv", index=False)

test_active_df.to_csv(OUTPUT_DATA_PATH / "test_active.csv", index=False)
train_active_df.to_csv(OUTPUT_DATA_PATH / "train_active.csv", index=False)

test_pv_df.to_csv(OUTPUT_DATA_PATH / "test_pv.csv", index=False)
train_pv_df.to_csv(OUTPUT_DATA_PATH / "train_pv.csv", index=False)


## Done!
We have 109 PV, active power and reactive power data sets as inputs. 

Now let's do the network models (in another notebook)